# Sentinel-1  geometric and radiometric terrain correction

In this tutorial you will learn how to generate a Sentinel-1 RTC (geometric and radiometric terrain corected) from a GRD Sentinel-1 images using [sarsen](https://github.com/bopen/sarsen).
 
The typical side-looking SAR system acquires with uniform sampling in azimuth and in slant rage, where the azimuth represents the time when a given target is acquired and the range represents the absolute sensor-to-target distance. 
Beacuse of this, the near range appears compressed with respect to far range. Furthermore, any deviation of the target elevation from a smooth geoid results in additional local geometric and radiometric distortions known as foreshortening and layover and shadow. Radar foreshortening is the effect of imaged terrain surfaces sloping towards the radar appearing shortened relative to those sloping away from the radar. Radar layover is an extreme case of foreshortening that occurs when the slope of the terrain is greater than the angle of the incident signal. 


### Terrain Correction (GTC)
The GRD Sentinel-1 product already provide a geometric corection that remove the compression effect on the near range, converting the data from slant-range to the ground range uniform sampling. The slant-range to ground-range corrections can be predicted if the imaging geometry is known, but the removal of the distortions due to target elevation requires an independent source of information, such as a digital elevantion model (DEM).  

The sarsen terrain correction convert a SAR image from range-azimuth to a terrestrial coordinate system (such as UTM or geodetic coordinates), as follows: 
- compute the image coordinates of each DEM point by resolving the zero-doppler equations  
- interpolate the image radiometry on the DEM pixels

The output is the input SAR image resampled on DEM coordinates.

### Radiometric Terrain Correction (RTC) 
Terrain variations do not affect only the position of a given point on the Earth's surface but also the brightness of the radar return. The sarsen radiometric terrain correction compensates the backscatter modulation generated by the topography of the scene in order to obtain a more uniform backscatter image emphasizing, thus, the radiometric differences of the terrain.
For the radiometric terrain correction sarsen implements the [Gamma Flatteining](https://ieeexplore.ieee.org/document/5752845) algorithm.



In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import logging

# logger = logging.basicConfig(level=logging.INFO)

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

## Install dependencies

In [ ]:
# !mamba install -c conda-forge -y make proj-data sentinelsat xmlschema

In [ ]:
# !pip install elevation sarsen xarray-sentinel

In [ ]:
import os

import rioxarray  # enable the `.rio` accessor
import xarray as xr

from sarsen import apps, scene

## Processing definition

In [ ]:
# create a temporary directory where to store downloaded data
os.makedirs("/tmp", exist_ok=True)

# DEM paths 
dem_urlpath = "/tmp/South-of-Redmond-10m.tif"
dem_10m_UTM_urlpath = dem_urlpath.strip(".tif") + "_UTM.tif"

# path to Sentinel-1 input product in the Planetary Computer
product_folder = "GRD/2021/12/17/IW/DV/S1B_IW_GRDH_1SDV_20211217T141304_20211217T141329_030066_039705_9048"

# band to be processed
measurement_group = "IW/VV"

## Download DEM of South-of-Redmond (Seattle, US) from Planetary Computer

In [ ]:
import adlfs
import planetary_computer
import pystac_client
import stackstac

#### Area of interest definition

In [ ]:
# seattle coordinates
longitude, latitude = [-121.95, 47.04]

# definition of area of interest
buffer = 0.2
bbox = [longitude - buffer, latitude - buffer, longitude + buffer, latitude + buffer]

#### Download all the DEMs defined on the area, possibly acquired in different times

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
threedep = catalog.get_child(id="3dep-seamless")

search = catalog.search(collections="3dep-seamless", bbox=bbox)
items = list(search.get_items())
items

In [ ]:
items_high_res = [
    planetary_computer.sign(item).to_dict()
    for item in items
    if item.properties["gsd"] == 10
]

dem_raster_all = stackstac.stack(items_high_res, bounds=bbox).squeeze()
dem_raster_all

#### Mean the DEMs along time and save it

In [ ]:
dem_raster = dem_raster_all.compute()
if "time" in dem_raster.dims:
    dem_raster = dem_raster.mean("time")
dem_raster.rio.set_crs(dem_raster_all.rio.crs)
dem_raster.rio.to_raster(dem_urlpath)

#### Convert the DEM in UTM

The DEM conversion in UTM is done to simplify the comparison between sarsen RTC with and the RTC provided on Planatery Computer.

In [ ]:
dem_UTM_corners = dict(x=slice(565000, 594000), y=slice(5220000, 5190000))
dem_raster = scene.open_dem_raster(dem_urlpath)

resolution = (10, 10)

# find the UTM zone
t_srs = dem_raster.rio.estimate_utm_crs()

# project in UTM
dem_raster_10m_UTM = dem_raster.rio.reproject(t_srs, resolution=resolution)

# crop DEM to our region of interest
dem_raster_10m_UTM = dem_raster_10m_UTM.sel(**dem_UTM_corners)

dem_raster_10m_UTM.rio.to_raster(dem_10m_UTM_urlpath)

In [ ]:
dem_raster_10m_UTM.plot();
plt.title("DEM in UTM coordinates");

## Download Sentinel-1 GRD from Planetary Computer

In [ ]:
import os

def mirror_folder(fs, storage_container, folder, exclude="vh"):
    for path, folders, files in fs.walk(f"{storage_container}/{folder}"):
        os.makedirs(path[len(storage_container) + 1 :], exist_ok=True)
        for f in files:
            if exclude in f:
                continue
            file_path = os.path.join(path, f)
            lfile_path = file_path[len(storage_container) + 1 :]
            if not os.path.isfile(lfile_path):
                print(file_path)
                fs.download(file_path, lfile_path + "~")
                os.rename(lfile_path + "~", lfile_path)

#### Find the target image in the Planetary Computer
- Connect to the Planetary Computer
- List the content of the Sentinel-1 folder product 

In [ ]:
grd_account_name = "sentinel1euwest"
grd_storage_container = "s1-grd"
grd_token = planetary_computer.sas.get_token(grd_account_name, grd_storage_container).token

grd_product_folder = f"{grd_storage_container}/{product_folder}"

grd_fs = adlfs.AzureBlobFileSystem(grd_account_name, credential=grd_token)
grd_fs.ls(grd_product_folder)

#### Download the Sentinel-1 product

In [ ]:
mirror_folder(grd_fs, grd_storage_container, product_folder)

#### Check downloaded product
Open Sentinel-1 GRD VV band with xarray-sentinel backend.

In [ ]:
ds = xr.open_dataset(
    product_folder,
    engine="sentinel-1",
    group="IW/VV",
    chunks={"slant_range_time": 2048},
)
ds

## Processing

#### GTC
Now we compute the geometric terrain correction.

With the `measurement_group` key you can define in the format {swath}/{polarizaton} (e.g. `"IW/VV"`) the band to be processed.

In [ ]:
gtc_path = os.path.basename(product_folder) + ".10m.GTC.tif"

apps.backward_geocode_sentinel1(
    product_urlpath = product_folder,
    measurement_group = measurement_group,
    dem_urlpath = dem_10m_UTM_urlpath,
    interp_method="nearest",
    chunks={"slant_range_time": 2048},
    output_urlpath=gtc_path,
)

gtc_path

In [ ]:
gtc = xr.open_dataarray(gtc_path).drop("band")
gtc.plot(vmax=0.4);

#### RTC
Here we compute the geometric and radiometric terrain correction.

In [ ]:
rtc_path = os.path.basename(product_folder) + ".10m.RTC.tif"

apps.backward_geocode_sentinel1(
    product_folder,
    measurement_group,
    dem_10m_UTM_urlpath,
    interp_method="nearest",
    correct_radiometry="gamma",
    output_urlpath=rtc_path,
    grouping_area_factor=(3, 3),
)

rtc_path

In [ ]:
rtc = xr.open_dataarray(rtc_path, cache=False).drop("band")
rtc.plot(vmax=0.4);

## Comparison between GTC and RTC

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 12))

gtc.plot(ax=axes[0], vmax=0.4)
axes[0].grid(c="red")

rtc.plot(ax=axes[1], vmax=0.4)
axes[1].grid(c="red")

plt.tight_layout()

## Comparison between sarsen RTC and Planetary Conputer RTC

#### Download RTC image from Planetary Computer

In [ ]:
rtc_account_name = "sentinel1euwestrtc"
rtc_storage_container = "sentinel1-grd-rtc"
rtc_token = planetary_computer.sas.get_token(rtc_account_name, rtc_storage_container).token

rtc_product_folder = f"{rtc_storage_container}/{product_folder}"

rtc_fs = adlfs.AzureBlobFileSystem(rtc_account_name, credential=rtc_token)
rtc_fs.ls(rtc_product_folder)

In [ ]:
mirror_folder(rtc_fs, rtc_storage_container, product_folder)

#### Open RTC image from Planetary

In [ ]:
rtc_pc = xr.open_dataarray(product_folder + "/measurement/iw-vv.rtc.tiff", cache=False).drop("band")
rtc_pc = rtc_pc.sel(dem_UTM_corners)
rtc_pc

#### Plot

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 12))

rtc_pc.plot(ax=axes[0], vmax=0.4)
axes[0].grid(c="red")

rtc.plot(ax=axes[1], vmax=0.4)
axes[1].grid(c="red")

plt.tight_layout()